In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
data = pd.read_csv('/Users/shemyakin/Downloads/materialy-k-zanyatiy-2019-25-06/adult.csv')

In [ ]:
data.head()

Видны непонятные значения в виде "?" знака. Попробуем заменить их на None, выявив таким образом пропуски в данных.

In [ ]:
data.info()

In [ ]:
data.replace(['?'], [None], inplace = True)

In [ ]:
data.isnull().sum()

Разделим наши данный на категориальные и числовые

In [ ]:
cat_columns = [c for c in data.columns if data[c].dtype.name == 'object']
num_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
print (cat_columns)
print (num_columns)

In [ ]:
data[cat_columns].describe()

In [ ]:
data[num_columns].describe()

Заменим пропуски в категориальных переменных на топовые (частовстречающиеся) значения 

In [ ]:
data_describe = data.describe(include=[object])
for c in cat_columns:
    data[c] = data[c].fillna(data_describe[c]['top'])


In [ ]:
data.isnull().sum()

Посмотрим числовые переменные

In [ ]:
import matplotlib.pyplot as plt

data.plot.bar()

Посмотрим какие значения у ключевого атрибута income

In [ ]:
data['income'].unique()

Для удобства заменим '>50K' на 1, а '<=50K' на 0

In [ ]:
data.replace('<=50K', 0, inplace = True)
             

In [ ]:
data.replace('>50K', 1, inplace = True)

In [ ]:
#chek our replace
data['income'].unique()

In [ ]:
binary    = [c for c in cat_columns if data_describe[c]['unique'] == 2]
nonbinary = [c for c in cat_columns if data_describe[c]['unique'] > 2]
print (binary, nonbinary)

In [ ]:
for c in binary[1:]:
    top = data_describe[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = 0
    data.loc[np.logical_not(top_items), c] = 1

In [ ]:
data_nonbinary = pd.get_dummies(data[nonbinary])
print (data_nonbinary.columns)


In [ ]:
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()


In [ ]:
data = pd.concat((data_numerical, data[binary_columns], data_nonbinary), axis=1)
data = pd.DataFrame(data)


In [ ]:
data.shape

In [ ]:
data.select_dtypes(exclude=['object']) \
           .join(pd.get_dummies(data.select_dtypes(include=['object']))).head(10)

Посмотрим корреляцию наших данных

In [ ]:
data.corr().style.background_gradient()

Разделим данные на Х и у. Т.к. доход у нас ключевой атрибут, то разделение будет выглядеть так:

In [ ]:
X = data.drop(('income'), axis=1)
y = data['income']


In [ ]:
X[:5]

In [ ]:

y[:7]

In [ ]:
X.corr().style.background_gradient('Spectral_r')


Разделим выборку на тренировочную и валидационную

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

In [ ]:
model = LogisticRegression()

from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)

err_train = np.mean(y_train != svc.predict(X_train))
err_test  = np.mean(y_test  != svc.predict(X_test))
print (err_train, err_test)

In [ ]:
model.fit(X_train, y_train )
y_pred = model.predict_proba( X_test )